In [1]:
import sys
import os 
import pandas as pd
import numpy as np
import json
import collections
import datetime as dt
import pytz

# NOTES
Time in use\
Mission started in last 30 days\
Missions have data (time %)\
Active Tags\
Webpage Use\
DELETE - Missions | Scans

# Team -> Budbee

In [2]:
team_id = 'ObjectId(5e39516344a3f400330cbd79)'
#Suomen Talotekniikka Jyväskylä ->'ObjectId(5d71fc4376d1ee2b2c27cf52)'
#MagnumMedical ->'ObjectId(5bb30fbf2669ca1a890025ac)'
#Budbee ->'ObjectId(5e39516344a3f400330cbd79)'

# Data 

In [3]:
teams = pd.read_csv('../Datasets/teams.csv')

In [4]:
missions = pd.read_csv('../Datasets/missions.csv')

In [5]:
scans = pd.read_csv('../Datasets/scan_events.csv')

In [6]:
loggers = pd.read_csv('../Datasets/loggers.csv')

In [8]:
users = pd.read_csv('../Datasets/users.csv')

In [60]:
#extracting user id
def user_extract(data):
    if 'user_id' in json.loads(data).keys():
        return 'ObjectId(' + list(json.loads(data)['user_id'].values())[0] +')'

#extracting teams data from request_logs
def team_extract(data):
    team_list = json.loads(data)['teams']
    clean_list = []
    if len(team_list) == 1:
        clean_list.append('ObjectId(' + team_list[0]['$oid'] + ')')
        return clean_list
    elif len(team_list) > 1:
        for team in team_list: 
            clean_list.append('ObjectId(' + team['$oid'] + ')')
        return clean_list
            
#flattening lists of lists from requests.teams
def flatten(aList):
    t = []
    for i in aList:
        if not isinstance(i, list):
              t.append(i)
        else:
            t.extend(flatten(i))
    return t

#extracting if team is in requests.teams
def team_check(data):
    if team_id in data:
        return 1 
    else:
        return 0
    
#requests time from now
def time_past(time):
    return dt.datetime.now().replace(tzinfo = pytz.UTC) - time.replace(tzinfo = pytz.UTC)

In [10]:
requests = pd.read_csv('../Datasets/request_logs.csv')
requests['user_id'] = requests['user'].apply(user_extract)
requests['teams'] = requests['user'].apply(team_extract)

In [11]:
requests['the_team'] = requests.loc[~requests['teams'].isnull(), 'teams'].apply(team_check)

In [12]:
requests['time'] = pd.to_datetime(requests['time'])

In [13]:
team_requests = requests[requests['the_team'] == 1]

In [14]:
team_requests['time_past'] = team_requests['time'].apply(time_past)

/Users/kristofersiimar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Merging

In [20]:
teams_b = teams.loc[teams['_id'] == team_id]
merged = loggers.merge(teams_b, how = 'inner', left_on = 'owner_id', right_on = '_id')
merged = merged.rename(columns = {'_id_x':'logger_id', '_id_y':'team_id',\
                                  'created_at_x':'logger_created_at', 'created_at_y':'team_created_at'})\
        .drop(columns = ['owner_id', 'serial', 'model'])

In [22]:
merged = merged.merge(missions, how = 'left', on = 'logger_id')
merged = merged.drop(columns = ['alert_rule_id', 'updated_at'])\
.rename(columns = {'created_at':'mission_created_at',\
                  'permissions_x':'logger_permissions', 'permissions_y':'mission_permissions'})

In [24]:
merged = merged.merge(scans, how = 'left', on = 'logger_id')
merged = merged.drop(columns = ['new_data', 'data_count', 'useragent', 'ip', 'name'])\
        .rename(columns = {'_id':'scan_id', 'created_at':'scan_created_at', 'location':'scan_location'})

In [33]:
merged['starts_at'] = pd.to_datetime(merged['starts_at'])
merged['ends_at'] = pd.to_datetime(merged['ends_at'])
merged['logger_created_at'] = pd.to_datetime(merged['logger_created_at'])

In [36]:
merged['mission_starts_past'] = merged['starts_at'].apply(time_past)

# Calculation

In [38]:
def ces(team_id, data, requests):
    score = 0 
    
    mission = len(data.loc[merged['mission_starts_past'] < pd.Timedelta('90 days')]['mission_id'].unique())
    active_user = len(requests.loc[requests['time_past'] <= pd.Timedelta('90 days')]['user_id'].unique())
    web_request = len(requests.loc[requests['time_past'] <= pd.Timedelta('90 days')]['_id'].unique())

    
    score += 34*(mission) + (19*scan) + (19*active_user) + (16.5*web_request) + (11.5*logger)
    print(('missions: ' + str(mission), 'active users: ' + str(active_user),\
           'web requests: ' + str(web_request)))

    
    return 'score: ' + str(score)

In [39]:
ces(team_id, merged, team_requests)

('missions: 25', 'active users: 7', 'web requests: 7519')


# Writing into csv

In [ ]:
requests.to_csv('datasets/Suomen Talotekniikka Jyväskylä_requests.csv', index = False)

In [ ]:
merged.to_csv('datasets/Suomen Talotekniikka Jyväskylä_mergedata.csv', index = False)